In [2]:
import torch
import timm
from PIL import Image
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sys

sys.path.append('../src')

print("Libraries imported successfully.")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "efficientnet_b0"  
IMAGE_SIZE = 224
BATCH_SIZE = 32            

print(f"Using device: {DEVICE}")


# num_classes=0 tells the model to output the feature vector before the final classification layer
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=0)
model = model.to(DEVICE)
model.eval() # Set the model to evaluation mode

# This resizes, converts to a tensor, and normalizes the images to match what the model was trained on
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print("Model and transforms are ready.")

Libraries imported successfully.
Using device: cpu
Model and transforms are ready.


In [3]:
def get_image_embeddings(image_paths):
    """
    Processes a list of image paths in batches and returns their embeddings.
    """
    all_embeddings = []
    # Process images in batches to manage memory usage
    for i in tqdm(range(0, len(image_paths), BATCH_SIZE), desc="Processing Image Batches"):
        batch_paths = image_paths[i:i + BATCH_SIZE]
        batch_images = []
        
        for path in batch_paths:
            try:
                # Open image and convert to RGB (important for consistency)
                img = Image.open(path).convert("RGB")
                img_tensor = transform(img)
                batch_images.append(img_tensor)
            except Exception as e:
                # If an image is broken or missing, we create a placeholder of zeros
                # This prevents the whole script from crashing
                print(f"Warning: Could not open {path}. Creating a zero placeholder. Error: {e}")
                batch_images.append(torch.zeros((3, IMAGE_SIZE, IMAGE_SIZE)))

        # Stack all image tensors in the batch into a single tensor
        batch_tensor = torch.stack(batch_images).to(DEVICE)
        
        # Get predictions from the model (no gradient calculation needed)
        with torch.no_grad():
            embeddings = model(batch_tensor)
        
        # Move embeddings to CPU and convert to a NumPy array before appending
        all_embeddings.append(embeddings.cpu().numpy())
        
    # Combine all the batch results into a single large NumPy array
    return np.vstack(all_embeddings)

In [ ]:

train_df = pd.read_csv("../data/raw/dataset/train.csv")
test_df = pd.read_csv("../data/raw/dataset/test.csv")
train_df['is_train'] = 1
test_df['is_train'] = 0
full_df = pd.concat([train_df, test_df], ignore_index=True)

# Helper function to get ONLY the filename from the URL
def get_filename_from_url(url):
    return os.path.basename(url)

# Apply the function to get a clean filename column
full_df['image_filename'] = full_df['image_link'].apply(get_filename_from_url)

# Build the correct, full paths to your downloaded images
train_image_paths = '../data/images/train/' + full_df[full_df['is_train'] == 1]['image_filename']
test_image_paths = '../data/images/test/' + full_df[full_df['is_train'] == 0]['image_filename']
all_image_paths = pd.concat([train_image_paths, test_image_paths]).tolist()

# --- THIS IS THE MAIN EXECUTION STEP ---
print("Starting image embedding generation... This will take some time.")
image_embeddings = get_image_embeddings(all_image_paths)
print(f"Embeddings generated with shape: {image_embeddings.shape}")

# Create a clean DataFrame from the embeddings
embedding_cols = [f"img_feat_{i}" for i in range(image_embeddings.shape[1])]
image_features_df = pd.DataFrame(image_embeddings, columns=embedding_cols)

# --- Save the results to a Parquet file. You will never have to run this again! ---
PROCESSED_DIR = "../data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True) # Ensure the directory exists
IMAGE_FEATURES_PATH = os.path.join(PROCESSED_DIR, "image_features_v1.parquet")
image_features_df.to_parquet(IMAGE_FEATURES_PATH, index=False)

print(f"\n IMAGE FEATURES SAVED to: {IMAGE_FEATURES_PATH}")

Starting image embedding generation... This will take some time.


Processing Image Batches:   0%|          | 7/4688 [00:22<4:13:42,  3.25s/it]

Processing Image Batches:   0%|          | 8/4688 [00:26<4:31:54,  3.49s/it]

Processing Image Batches:   0%|          | 9/4688 [00:28<4:17:24,  3.30s/it]

Processing Image Batches:   0%|          | 10/4688 [00:31<4:11:07,  3.22s/it]

...

Processing Image Batches:  66%|██████▌   | 3088/4688 [3:01:02<24:34,  1.09it/s]

Processing Image Batches:  66%|██████▌   | 3089/4688 [3:01:02<23:07,  1.15it/s]

Processing Image Batches:  66%|██████▌   | 3090/4688 [3:01:04<25:54,  1.03it/s]

Processing Image Batches:  66%|██████▌   | 3091/4688 [3:01:05<27:25,  1.03s/it]

Processing Image Batches:  66%|██████▌   | 3096/4688 [3:01:18<59:23,  2.24s/it]  